## Generar imágenes MEDIA, BAJA y MINI

Este script genera imágenes en MEDIA y BAJA a partir de ALTA.  El script espera una tabla con la columna 'ruta_doc', que contiene la rutas a la carpetas de documentos del archivo.  Tambien espera la columna 'doc_existe' con 0 y 1, representando la exsitencia o no de la carpeta de documento.

In [ ]:
import os
import sys
import datetime
from pathlib import Path
import numpy as np
import pandas as pd
from PIL import Image

In [ ]:
size_media = (1920, 1920)
size_baja = (512, 512)
size_mini = (128, 128)

# Lista de prefijos y sufijos para filtar archivos
prefijos_prohibidos = ['.', 'DS_']
sufijos_permitidos = ['.tiff','.tif','.TIF','.tif','.png','.PNG','.jpg','.JPG','.jpeg','.JPEG','.RAW','.CRW']
modos = ['I;16', 'I;16B']

## Función para filtrar archivos según su extension

Esta función usa la lista de prefijos y sufijos para filtrar los archivos.

In [ ]:
def filtrar_archivos(arr, sufijos_incluidos, prefijos_excluidos):
    def has_allowed_suffix(s, suffixes):
        return any(s.endswith(suffix) for suffix in suffixes)
    
    def has_not_allowed_prefix(s, prefixes):
        return any(s.startswith(prefix) for prefix in prefixes)
    
    filtered_elements = [
        element for element in arr 
        if has_allowed_suffix(element.suffix, sufijos_incluidos) and not has_not_allowed_prefix(element.name, prefijos_excluidos)
    ]
    
    return filtered_elements

In [ ]:
#ruta a la tabla de datos
tabla = "tabla.csv"
#nombre de la columna con las rutas a la carpeta de documentos
col_rutas = "ruta_doc"


## Limpieza básica

Elimina filas vacías y filtra los documentos que existen solamente.

In [ ]:
datos = pd.read_csv(tabla)
datos = datos.dropna(how='all')
datos = datos[datos['doc_existe'] == 1]
datos

## Crear derivados para MEDA, BAJA y MINI

In [ ]:
for index, row in datos.iterrows():
    
    ruta_doc = Path(row[col_rutas])
    ruta_alta = Path(ruta_doc) / Path('ALTA')
    ruta_media = Path(ruta_doc) / Path('MEDIA')
    ruta_baja = Path(ruta_doc) / Path('BAJA')
    ruta_mini = Path(ruta_doc) / Path('MINI')

    archivos_orig = filtrar_archivos([f for f in ruta_alta.iterdir()], sufijos_permitidos, prefijos_prohibidos)

    print(archivos_orig)

    ## Generar Derivados
    for orig in archivos_orig:

        salida_media = ruta_media / Path(f"{orig.stem}.jpg")
        salida_baja = ruta_baja / Path(f"{orig.stem}.jpg")
        salida_mini = ruta_mini / Path(f"{orig.stem}.jpg")

        #inim = None
        #rgbim = None

        try:
            inim = Image.open(orig)
            # Solución temporal para error en conversión de I,16 a RGB
            if inim.mode in modos:
	            #inim = inim.point(lambda i: i * (1 / 255))
                inim = inim.convert("F").point(lambda i: i * (1 / 255))
            
            rgbim = inim.convert('RGB')

        except Exception as e:
            print(f"Error abriendo: {orig}")
            print(e)
            continue
        
        ## crear media

        try:
            media_im = rgbim.copy()
            media_im.thumbnail(size_media)
            media_im.save(salida_media, "jpeg")
        except OSError as e:
            print(f"Error creando MEDIA: {salida_media}: {e}")
            continue
        
        print(f"Creada MEDIA: {salida_media}")

        # Crear bajas
        try:
            baja_im = rgbim.copy()
            baja_im.thumbnail(size_baja)
            baja_im.save(salida_baja, "jpeg")
        except OSError as e:
            print(f"Error creando BAJA: {salida_baja}: {e}")
            continue
        
        print(f"Creada BAJA: {salida_baja}")

        try:
            mini_im = rgbim.copy()
            mini_im.thumbnail(size_mini)
            mini_im.save(salida_mini, "jpeg")
        except OSError as e: 
            print(f"Error crea  ndo MINI: {salida_mini}: {e}")
            continue
        
        print(f"Creada MINI: {salida_mini}")